# Apache Spark

Сегодня будет говорить про Apache Spark - более удобный фреймворк для обработки больших данных на базе Hadoop.

С Spark можно работать из ноутбуков в Data Sphere, но так как нам еще потребуется запускать bash команды, то я буду запускать все команды ниже из ноутбука на мастер-ноде

PySpark - это не обычная библиотека, поэтому по-умолчанию ее нет в списке установленных пакетов

Чтобы решить эту проблему простым способом, добрые люди сделали небольшую библиотеку findspark

In [1]:
! pip install findspark

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName="lsml-app")

# Работаем с RDD

RDD - это базовый строительный блок для Spark. Спарк внимательно следит за тем, где лежат части RDD и как они были созданы. RDD по сути своей представляют упорядоченный набор записей. Большое число функций считают, что это пары ключ-значение (также как было в MapReduce), но на деле это может быть и произвольные данные.

RDD сами по себе неизменяемые. Можно лишь получить новый RDD, применяя различные операции к изначальному RDD.

Существуют два вида операций над RDD - Действия (actions) и Трансформации (transformations).

Трансформации не применяются сразу - они лишь записываются в пул примененных операций. Чтобы что-то действительно началось считаться, нужно применить уже действие - тогда все указанные транформации действительно начнут считаться на кластере.

Давайте сразу смотреть на примерах, чтобы стало понятно.

In [4]:
rdd = sc.parallelize(range(10))  # Создаем rdd из обычного списка

In [5]:
rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [6]:
rdd.collect()  # Получить значение всего RDD в память. Аккуратнее - если RDD большой, у вас просто лопнем питон

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
rdd.count()  # Считаем количество элементов в RDD

10

In [8]:
rdd.first()  # Берем только первый элемент

0

In [9]:
rdd.take(2)  # Берем первые N элементов

[0, 1]

In [10]:
rdd.mean()  # Считаем среднее по всем элементам. Важно, чтобы элементы внутри RDD поддерживали суммирование и деление

4.5

In [11]:
rdd = sc.parallelize(["biba", "kuka"])  # Можем положить и строки

In [12]:
rdd.collect()

['biba', 'kuka']

In [13]:
! hdfs dfs -ls /user

Found 4 items
drwxr-xr-x   - hive   hadoop          0 2022-01-30 14:30 /user/hive
drwxr-xr-x   - ubuntu hadoop          0 2022-02-05 09:11 /user/spark-example
drwxr-xr-x   - ubuntu hadoop          0 2022-02-05 09:25 /user/tweets
drwxr-xr-x   - ubuntu hadoop          0 2022-02-05 08:15 /user/ubuntu


In [14]:
! hdfs dfs -mkdir -p /user/spark-example

In [15]:
! hdfs dfs -rm -r /user/spark-example/biba_and_kuka.txt || true

Deleted /user/spark-example/biba_and_kuka.txt


In [16]:
rdd.saveAsTextFile("/user/spark-example/biba_and_kuka.txt")  # Сохраняем RDD в HDFS

In [17]:
! hdfs dfs -ls /user/spark-example/biba_and_kuka.txt

Found 3 items
-rw-r--r--   1 ubuntu hadoop          0 2022-02-05 20:03 /user/spark-example/biba_and_kuka.txt/_SUCCESS
-rw-r--r--   1 ubuntu hadoop          5 2022-02-05 20:03 /user/spark-example/biba_and_kuka.txt/part-00000
-rw-r--r--   1 ubuntu hadoop          5 2022-02-05 20:03 /user/spark-example/biba_and_kuka.txt/part-00001


In [18]:
! hdfs dfs -cat /user/spark-example/biba_and_kuka.txt/*

biba
kuka


Добавим теперь еще трансформации

In [19]:
rdd = sc.parallelize(range(10))  # Создаем rdd из обычного списка

In [20]:
# Создаем rdd в котором каждый элемент возведен в квадрат
# map работает примерно также как и map в MapReduce. 
# Разница - мы не обрабатываем блок самостоятельно, а пишем функцию для обработки ровно одной записи
squares = rdd.map(lambda x: x**2).map(lambda x: x + 1)

# ВАЖНО - на самом деле ничего считаться в этот момент не начало
# Мы лишь записали наше желание получить новый RDD и записали это желание в squares

In [21]:
squares.first() 

# Так как мы применили Action то вот теперь все трансформации запустились
# Но так как action требует только первую строку, то Spark оптимизировал вычисления
# он прочитал только первую строку и для нее вычислил значение

1

In [22]:
squares.collect()

[1, 2, 5, 10, 17, 26, 37, 50, 65, 82]

#### Начнем работать с данными

Датасет - тот же, что и на предыдущем семинаре

In [23]:
! hdfs dfs -ls /user/tweets/data 

Found 13 items
-rw-r--r--   1 ubuntu hadoop   94371561 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_1.csv
-rw-r--r--   1 ubuntu hadoop   94371615 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_10.csv
-rw-r--r--   1 ubuntu hadoop   94371552 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_11.csv
-rw-r--r--   1 ubuntu hadoop   94371703 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_12.csv
-rw-r--r--   1 ubuntu hadoop    8238864 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_13.csv
-rw-r--r--   1 ubuntu hadoop   94371748 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_2.csv
-rw-r--r--   1 ubuntu hadoop   94371796 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_3.csv
-rw-r--r--   1 ubuntu hadoop   94371606 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_4.csv
-rw-r--r--   1 ubuntu hadoop   94371616 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_5.csv
-rw-r--r--   1 ubuntu hadoop   94371646 2022-01-31 13:53 /user/tweets/data/IRAhandle_tweets_6.csv
-

In [24]:
data = sc.textFile("/user/tweets/data/*")

In [25]:
data.first()

'906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on trial for corruption and you\'ve barely heard a peep from the mainstream media."" ~ @nedryun https://t.co/gh6g0D1oiC",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses/914580356430536707,https://twitter.com/10_gop/status/914580356430536707/video/1,,'

In [26]:
import csv

def extract_text(raw_string):
    parsed_line = next(csv.reader([raw_string]))
    text = parsed_line[2]
    return text

In [27]:
data.map(extract_text).first()

'"We have a sitting Democrat US Senator on trial for corruption and you\'ve barely heard a peep from the mainstream media." ~ @nedryun https://t.co/gh6g0D1oiC'

In [28]:
import re

def extract_words(text):
    pattern = re.compile(r"[a-z]+")
    result = []
    for match in pattern.finditer(text.lower()):
        word = match.group(0)
        result.append(word)
    return result

In [29]:
data.map(extract_text).map(extract_words).take(2)

[['we',
  'have',
  'a',
  'sitting',
  'democrat',
  'us',
  'senator',
  'on',
  'trial',
  'for',
  'corruption',
  'and',
  'you',
  've',
  'barely',
  'heard',
  'a',
  'peep',
  'from',
  'the',
  'mainstream',
  'media',
  'nedryun',
  'https',
  't',
  'co',
  'gh',
  'g',
  'd',
  'oic'],
 ['marshawn',
  'lynch',
  'arrives',
  'to',
  'game',
  'in',
  'anti',
  'trump',
  'shirt',
  'judging',
  'by',
  'his',
  'sagging',
  'pants',
  'the',
  'shirt',
  'should',
  'say',
  'lynch',
  'vs',
  'belt',
  'https',
  't',
  'co',
  'mlh',
  'i',
  'lzz']]

In [30]:
data.map(extract_text).flatMap(extract_words).first()

'we'

In [31]:
data.map(extract_text).flatMap(extract_words).take(10)

['we',
 'have',
 'a',
 'sitting',
 'democrat',
 'us',
 'senator',
 'on',
 'trial',
 'for']

Все трансформации вычисляются каждый раз с самого первого RDD. Чтобы уменьшить количество лишних вычислений можно закешировать временный результат. Тогда он будет по максимуму переиспользоваться.

In [32]:
words = data.map(extract_text).flatMap(extract_words).cache()

In [33]:
words.count()

41946754

In [34]:
words.count()

41946754

На моем запуске второй запуск `words.count()` работал 2 секунды вместо 17

#### Word count

Попробуем реализовать тот же алгоритм, что и для классического MapReduce

In [35]:
words.map(lambda x: (x, 1)).first()  # Строим пары ключ значение

('we', 1)

In [36]:
(
    words
    .map(lambda x: (x, 1))
    .groupByKey()  # Функция работает, только если RDD - это пары ключ-значение
    .take(2)
)

[('ioam', <pyspark.resultiterable.ResultIterable at 0x7f6150197700>),
 ('mgkdv', <pyspark.resultiterable.ResultIterable at 0x7f6150197c40>)]

In [37]:
(
    words
    .map(lambda x: (x, 1))
    .groupByKey()  # Функция работает, только если RDD - это пары ключ-значение
    .map(lambda x: (x[0], sum(list(x[1]))))
    .take(10)
)

[('arrives', 910),
 ('belt', 502),
 ('navy', 2173),
 ('protests', 3288),
 ('j', 57058),
 ('this', 118604),
 ('much', 13829),
 ('said', 13532),
 ('two', 17046),
 ('lying', 3397)]

In [38]:
(
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)  # Если уже готовая функция для reduce
    .take(10)
)

[('arrives', 910),
 ('belt', 502),
 ('navy', 2173),
 ('protests', 3288),
 ('j', 57058),
 ('this', 118604),
 ('much', 13829),
 ('said', 13532),
 ('two', 17046),
 ('lying', 3397)]

In [39]:
(
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(10, lambda x: -x[1])  # Сортируем по значению функции
)

[('t', 3015051),
 ('co', 2833375),
 ('https', 2454132),
 ('the', 591885),
 ('to', 589004),
 ('in', 457433),
 ('a', 412888),
 ('s', 397889),
 ('http', 375299),
 ('of', 350983)]

In [40]:
result_50 = (
    words
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(50, lambda x: -x[1])
)

stop_words = [word for word, _ in result_50]  # Предподсчитали стоп слова

In [41]:
stop_words

['t',
 'co',
 'https',
 'the',
 'to',
 'in',
 'a',
 's',
 'http',
 'of',
 'i',
 'for',
 'and',
 'is',
 'on',
 'you',
 'trump',
 'news',
 'it',
 'with',
 'at',
 'that',
 'this',
 'rt',
 'm',
 'are',
 'be',
 'u',
 'my',
 'not',
 'we',
 'by',
 'from',
 'd',
 'your',
 'as',
 'new',
 'r',
 'have',
 'all',
 'n',
 'k',
 'he',
 'will',
 'f',
 'w',
 'was',
 'after',
 'who',
 'they']

In [42]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .takeOrdered(50, lambda x: -x[1])
)

[('just', 63572),
 ('what', 62512),
 ('c', 62430),
 ('e', 62022),
 ('b', 61064),
 ('about', 60260),
 ('police', 59042),
 ('up', 58757),
 ('l', 58478),
 ('can', 58178),
 ('g', 57828),
 ('o', 57769),
 ('p', 57638),
 ('man', 57610),
 ('j', 57058),
 ('x', 56777),
 ('h', 56523),
 ('y', 56216),
 ('no', 55914),
 ('out', 55858),
 ('v', 55327),
 ('people', 55169),
 ('me', 54160),
 ('but', 53036),
 ('sports', 52223),
 ('so', 52082),
 ('if', 51593),
 ('obama', 50655),
 ('z', 50303),
 ('q', 50137),
 ('his', 49153),
 ('us', 47572),
 ('world', 47475),
 ('how', 46947),
 ('get', 46941),
 ('like', 46806),
 ('more', 46580),
 ('has', 45670),
 ('do', 45175),
 ('an', 44963),
 ('now', 44180),
 ('politics', 44031),
 ('don', 43979),
 ('when', 43894),
 ('amp', 43392),
 ('one', 43370),
 ('our', 43246),
 ('over', 42735),
 ('workout', 42437),
 ('black', 40133)]

Кроме базовых, есть еще и много продвинутых сложных функций
Например можем посчитать уникальные слова в датасете

Список всех можно смотреть в документации

https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions

https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

In [43]:
words.distinct().take(10)

['arrives',
 'belt',
 'navy',
 'protests',
 'j',
 'this',
 'much',
 'said',
 'two',
 'lying']

In [44]:
words.distinct().count()

2831736

Однако иногда каких-то базовых примитивов может и не найтись. Например для RDD нет функции `limit` или около того.

Поэтому чтобы решить задачу top10 и сохранить это в HDFS нужно применить некоторую изобретательность

In [45]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .take(10)
)

[((63572, 'just'), 0),
 ((62512, 'what'), 1),
 ((62430, 'c'), 2),
 ((62022, 'e'), 3),
 ((61064, 'b'), 4),
 ((60260, 'about'), 5),
 ((59042, 'police'), 6),
 ((58757, 'up'), 7),
 ((58478, 'l'), 8),
 ((58178, 'can'), 9)]

In [46]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .map(lambda x: x[0])
    .collect()
)

[(63572, 'just'),
 (62512, 'what'),
 (62430, 'c'),
 (62022, 'e'),
 (61064, 'b'),
 (60260, 'about'),
 (59042, 'police'),
 (58757, 'up'),
 (58478, 'l'),
 (58178, 'can')]

In [47]:
! hdfs dfs -ls /user/tweets/

Found 10 items
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 13:53 /user/tweets/data
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:31 /user/tweets/lang-dist
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:35 /user/tweets/mistake
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:00 /user/tweets/result
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:14 /user/tweets/result-fast1
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:17 /user/tweets/result-fast2
drwxr-xr-x   - ubuntu hadoop          0 2022-02-05 09:25 /user/tweets/spark
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:04 /user/tweets/top10
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:23 /user/tweets/top10-fast
drwxr-xr-x   - ubuntu hadoop          0 2022-01-31 14:10 /user/tweets/top10-stop-words


In [48]:
! hdfs dfs -rm -r /user/tweets/spark/top10 || true

Deleted /user/tweets/spark/top10


In [49]:
(
    words
    .filter(lambda x: x not in stop_words)
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .map(lambda x: (x[1], x[0]))
    .sortByKey(ascending=False)
    .zipWithIndex()
    .filter(lambda x: x[1] < 10)
    .map(lambda x: x[0])
    .saveAsTextFile('/user/tweets/spark/top10')
)

In [50]:
! hdfs dfs -cat /user/tweets/spark/top10/*

(63572, 'just')
(62512, 'what')
(62430, 'c')
(62022, 'e')
(61064, 'b')
(60260, 'about')
(59042, 'police')
(58757, 'up')
(58478, 'l')
(58178, 'can')


#### Partitions

Под капотом Spark эксплуатирует примерно те же идеи, что и классический MapReduce. Это означает, что при необходимости сортировки, он разбивает ключи на группы и передает редюсерам на обработку только их часть.

На этот процесс также можно влиять. Это может позводить улучшить производительность программ, а также решить проблемы переполнения редюсеров.

In [51]:
words.getNumPartitions()

13

In [52]:
numbers = sc.parallelize(range(10))

In [53]:
numbers.glom().collect()  # Получаем доступ до данных в каждей партиции

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

In [54]:
squares = numbers.map(lambda x: (x, x**2))

Операции изменения партиций предполагают наличие ключа, поэтому вначале преобразуем данные к виду ключ-значение

In [55]:
squares.partitionBy(2).glom().collect()

[[(2, 4), (8, 64), (6, 36), (0, 0), (4, 16)],
 [(9, 81), (7, 49), (3, 9), (1, 1), (5, 25)]]

In [56]:
squares.partitionBy(15).glom().collect()

[[(0, 0)],
 [(1, 1)],
 [(2, 4)],
 [(3, 9)],
 [(4, 16)],
 [(5, 25)],
 [(6, 36)],
 [(7, 49)],
 [(8, 64)],
 [(9, 81)],
 [],
 [],
 [],
 [],
 []]

In [57]:
def custom_partitioner(value):
    return value % 3

In [58]:
squares.partitionBy(3, custom_partitioner).glom().collect()

[[(6, 36), (3, 9), (0, 0), (9, 81)],
 [(4, 16), (7, 49), (1, 1)],
 [(5, 25), (8, 64), (2, 4)]]

Таким образом можно выбирать более удачные способы разбиения и например увеличивать количество редюсеров под вашу задачу.

Или наоборот, уменьшать количество количество партиций, если они избыточны. Например вы отфильтровали гигантский датасет и теперь вам больше не требуется такое гигантское количество партиций для работы.

Для этого можно использовать и `repartition` как делали выше, однако этот метод запустит пересортировку вообще всего RDD, что дорого и излишне. Чтобы так не было, можно использовать функцию `coalesce` - она просто схлопнуть вместе те партиции, котороые уже находятся на одной машине, что значительно уменьшит количество лишних телодвижений.

In [59]:
squares.glom().collect()

[[(0, 0)],
 [(1, 1)],
 [(2, 4)],
 [(3, 9)],
 [(4, 16)],
 [(5, 25)],
 [(6, 36)],
 [(7, 49)],
 [(8, 64)],
 [(9, 81)]]

In [60]:
squares.filter(lambda x: x[0] >= 7).glom().collect()

[[], [], [], [], [], [], [], [(7, 49)], [(8, 64)], [(9, 81)]]

In [61]:
squares.filter(lambda x: x[0] >= 7).coalesce(2).glom().collect()

[[], [(7, 49), (8, 64), (9, 81)]]

#### DataFrame и SQL

Уже текущий набор функций - это большой шаг вперед относительно классического MapReduce. Однако на этом плюшки Spark не заканчиваются. Разработчики пошли дальше и начали внедрять еще более высокоуровневый интерфейс для работы с данными, который может сильно упростить жизнь разработчикам.

DataFrame - это модель таблицы, построенная поверх RDD. О ней можно думать как о Pandas на стероидах.

In [62]:
rdd = sc.parallelize([("a", 1), ("a", 2), ("b", 3), ("b", 4)])
rdd.collect()

[('a', 1), ('a', 2), ('b', 3), ('b', 4)]

In [63]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [64]:
df = se.createDataFrame(rdd)
df.printSchema()
df.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  a|  1|
|  a|  2|
|  b|  3|
|  b|  4|
+---+---+



In [65]:
df = se.createDataFrame(
    rdd.map(lambda x: Row(pipa=x[0], pupa=x[1]))
)
df.printSchema()
df.show()

root
 |-- pipa: string (nullable = true)
 |-- pupa: long (nullable = true)

+----+----+
|pipa|pupa|
+----+----+
|   a|   1|
|   a|   2|
|   b|   3|
|   b|   4|
+----+----+



Для удобства есть встроенные функции конвертации в pandas и оттуда

In [66]:
pandas_df = df.toPandas()
pandas_df

,pipa,pupa
0,a,1
1,a,2
2,b,3
3,b,4


In [67]:
df = se.createDataFrame(pandas_df)
df.printSchema()
df.show()

root
 |-- pipa: string (nullable = true)
 |-- pupa: long (nullable = true)

+----+----+
|pipa|pupa|
+----+----+
|   a|   1|
|   a|   2|
|   b|   3|
|   b|   4|
+----+----+



Есть специальные функции, которые умеют работать с популярными форматами хранения таблиц, и строить их в HDFS.

Прочтем нашу таблицу через DataFrame

In [72]:
df = se.read.csv('/user/tweets/data/*', header=False, inferSchema=True)

In [73]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)



In [74]:
columns = [
    'external_author_id',
    'author',
    'content',
    'region',
    'language',
    'publish_date',
    'harvested_date',
    'following',
    'followers',
    'updates',
    'post_type',
    'account_type',
    'retweet',
    'account_category',
    'new_june_2018',
    'alt_external_id',
    'tweet_id',
    'article_url',
    'tco1_step1',
    'tco2_step1',
    'tco3_step1'
]
df = df.toDF(*columns)
df.printSchema()

root
 |-- external_author_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- content: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- harvested_date: string (nullable = true)
 |-- following: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- updates: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- retweet: string (nullable = true)
 |-- account_category: string (nullable = true)
 |-- new_june_2018: string (nullable = true)
 |-- alt_external_id: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- article_url: string (nullable = true)
 |-- tco1_step1: string (nullable = true)
 |-- tco2_step1: string (nullable = true)
 |-- tco3_step1: string (nullable = true)



In [75]:
df.show()

+------------------+---------------+--------------------+--------------------+-------------+---------------+---------------+--------------+---------+-------+---------+------------+-------+----------------+-------------+---------------+------------------+--------------------+--------------------+--------------------+----------+
|external_author_id|         author|             content|              region|     language|   publish_date| harvested_date|     following|followers|updates|post_type|account_type|retweet|account_category|new_june_2018|alt_external_id|          tweet_id|         article_url|          tco1_step1|          tco2_step1|tco3_step1|
+------------------+---------------+--------------------+--------------------+-------------+---------------+---------------+--------------+---------+-------+---------+------------+-------+----------------+-------------+---------------+------------------+--------------------+--------------------+--------------------+----------+
|        1647

In [76]:
df[['author', 'content']].show()

+---------------+--------------------+
|         author|             content|
+---------------+--------------------+
|CARRIETHORNTHON|New Study Reveals...|
|CARRIETHORNTHON|Lindsey Graham ha...|
|CARRIETHORNTHON|. @LindseyGrahamS...|
|CARRIETHORNTHON|2016 Power Index:...|
|CARRIETHORNTHON|I self identify a...|
|CARRIETHORNTHON|.@UW varsity eigh...|
|CARRIETHORNTHON|Since the 1980s a...|
|CARRIETHORNTHON|Records: Wife Sus...|
|CARRIETHORNTHON|"""It will create...|
|CARRIETHORNTHON|The good news abo...|
|CARRIETHORNTHON|"""Culture fit"" ...|
|CARRIETHORNTHON|Things to Know Ab...|
|CARRIETHORNTHON|This is what happ...|
|CARRIETHORNTHON|#FloridaMan throw...|
|CARRIETHORNTHON|Report: DHS 'Red-...|
|CARRIETHORNTHON|More proof that T...|
|CARRIETHORNTHON|World Naked Bike ...|
|CARRIETHORNTHON|#Business — New @...|
|CARRIETHORNTHON|Pre-Game warm up ...|
|CARRIETHORNTHON|#News — Retired #...|
+---------------+--------------------+
only showing top 20 rows



In [77]:
df.registerTempTable('tweets')  # Регистрируем как временную таблицу для SQL

In [78]:
se.sql("""
    SELECT author, content, followers
    FROM tweets
    WHERE followers > 100
    LIMIT 10
""").show()

+---------------+--------------------+---------+
|         author|             content|followers|
+---------------+--------------------+---------+
|CARRIETHORNTHON|New Study Reveals...|      207|
|CARRIETHORNTHON|Lindsey Graham ha...|      207|
|CARRIETHORNTHON|. @LindseyGrahamS...|      207|
|CARRIETHORNTHON|2016 Power Index:...|      207|
|CARRIETHORNTHON|I self identify a...|      207|
|CARRIETHORNTHON|.@UW varsity eigh...|      207|
|CARRIETHORNTHON|Since the 1980s a...|      207|
|CARRIETHORNTHON|Records: Wife Sus...|      207|
|CARRIETHORNTHON|"""It will create...|      207|
|CARRIETHORNTHON|The good news abo...|      207|
+---------------+--------------------+---------+



In [79]:
se.sql("""
    SELECT language, count(*) as tw_count
    FROM tweets
    WHERE followers > 100
    GROUP BY language
""").show()

+------------------+--------+
|          language|tw_count|
+------------------+--------+
|              Urdu|      49|
|          Malaysia|      27|
|           Turkish|     373|
|              Iraq|     275|
|           Germany|     190|
|       Afghanistan|      35|
|           Kannada|       1|
|             Malay|     216|
|           Finnish|     520|
|              Thai|      33|
|            France|      11|
|         Icelandic|     455|
|            Pushto|     309|
|            Somali|     238|
|        Indonesian|     155|
|LANGUAGE UNDEFINED|    8143|
|              null|     157|
|         Ukrainian|   34620|
|Tagalog (Filipino)|     215|
|     United States|   14809|
+------------------+--------+
only showing top 20 rows



In [80]:
top5_lang = se.sql("""
    SELECT language, count(*) as tw_count
    FROM tweets
    WHERE followers > 100
    GROUP BY language
    ORDER BY tw_count DESC
    LIMIT 5
""")
top5_lang.show()

+-------------+--------+
|     language|tw_count|
+-------------+--------+
|      English| 1913019|
|      Russian|  546315|
|       German|   61941|
|    Ukrainian|   34620|
|United States|   14809|
+-------------+--------+



In [81]:
only_langs_df = se.sql("""
    SELECT language
    FROM (
        SELECT language, count(*) as tw_count
        FROM tweets
        WHERE followers > 100
        GROUP BY language
        ORDER BY tw_count DESC
        LIMIT 5
    )
""")
only_langs_df.show()

+-------------+
|     language|
+-------------+
|      English|
|      Russian|
|       German|
|    Ukrainian|
|United States|
+-------------+



In [82]:
only_langs_df.registerTempTable('languages')

In [83]:
se.sql("""
    SELECT author, language
    FROM tweets
    WHERE language in (SELECT * FROM languages)
    LIMIT 10
""").show()

+---------------+--------+
|         author|language|
+---------------+--------+
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
+---------------+--------+



In [84]:
se.sql("""
    SELECT author, t.language
    FROM tweets t
        inner join languages l on l.language = t.language
    LIMIT 10
""").show()

+---------------+--------+
|         author|language|
+---------------+--------+
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
|CARRIETHORNTHON| English|
+---------------+--------+



In [85]:
# Из под датафрейма всегда можно вынуть RDD и работать напрямую уже с ним

top5_lang.rdd.map(lambda x: x.language.upper()).collect()

['ENGLISH', 'RUSSIAN', 'GERMAN', 'UKRAINIAN', 'UNITED STATES']

In [86]:
top5_lang.collect()

[Row(language='English', tw_count=1913019),
 Row(language='Russian', tw_count=546315),
 Row(language='German', tw_count=61941),
 Row(language='Ukrainian', tw_count=34620),
 Row(language='United States', tw_count=14809)]